table containing information on consumers and their expenditure on various kinds of products.

identify the top two highest-grossing products in each category and output the category, product, and total spend information.

In [47]:
import pandas as pd

# create data-frame

product_spend_df = pd.DataFrame({'category': ['appliance','appliance','appliance','electronics','electronics','electronics'],
                    'product': ['refrigerator','refrigerator','washing machine','vacuum', 'wireless headset', 'vacuum'],
                   'user_id' : [165,123,123,178,156,145],
                    'spend' : [246,299.99,219.8,152,249.9,189],
                    'transaction_date' : ['12/26/2022','3/2/2022','3/2/2022','4/5/2022','7/8/2022', '7/15/2022']
                  })

In [48]:
import sqlite3

# create connection to in memory sqlite db
conn = sqlite3.connect(":memory:")

# save prices dataframe to sqlite db
product_spend_df.to_sql(name="product_spend", con=conn, index=False)

Build a named sub query result set of data that will contain information about the total amount spent on each product in each category 

rank the products in each category in descending order based on the total amount spent by consumers to acquire them. 
the product with the highest spend should come first, followed by the others in each category.

apply the Rank() Window Function across each category of products and rank the products in descending order of 
total amount spent.

At the end  filter for a rank less than or equal to 2 to get two highest-grossing products

In [49]:
query = """
  
with category_wise_spend_analysis AS ( 
    SELECT category,
           product,
           sum(spend) AS total_spend
    FROM product_spend
    GROUP BY
     category, product
   ),
   category_wise_top_spend AS (
   SELECT *, 
   RANK() OVER (PARTITION BY category ORDER BY category, total_spend DESC) AS ranking
   FROM category_wise_spend_analysis
   )
    
select *
from category_wise_top_spend
WHERE 
  ranking <=2
ORDER BY 
  category, ranking
    
"""


In [50]:
# use pandas read_sql to execute the query and return a dataframe
result_df = pd.read_sql(query, con=conn)

In [51]:
result_df

,category,product,total_spend,ranking
0,appliance,refrigerator,545.99,1
1,appliance,washing machine,219.80,2
2,electronics,vacuum,341.00,1
3,electronics,wireless headset,249.90,2
